In [85]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

SAVE_PATH = './regensburg_pediatric_appendicitis.csv'  

try:
    data = pd.read_csv(SAVE_PATH)
except:
    from ucimlrepo import fetch_ucirepo 
    regensburg_pediatric_appendicitis = fetch_ucirepo(id=938) 
    data_a = regensburg_pediatric_appendicitis['data']['features']
    data_b = regensburg_pediatric_appendicitis['data']['targets']
    data = pd.concat([data_a, data_b ], axis=1)
    data.to_csv(SAVE_PATH, index=False)


In [ ]:
categorical_cols = data.select_dtypes(include=['object', 'category']).columns

# remove targets column
categorical_cols = categorical_cols.drop('Diagnosis')
categorical_cols = categorical_cols.drop('Management')
categorical_cols = categorical_cols.drop('Severity')

data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

#convert true false to 1 0
data_encoded = data_encoded.replace({True: 1, False: 0})

display(data_encoded)

# Variable Selection

Numerical Variables

In [ ]:
numerical_features = data_encoded.select_dtypes(include=['float64', 'int64']).columns
display(numerical_features)

# sns.kdeplot(data=data_encoded[numerical_features], label = numerical_features)


In [ ]:
#Not normally distributed -- Use Mann-Whitney U test
from scipy.stats import mannwhitneyu

appendicitis = data_encoded[data_encoded['Diagnosis'] == 'appendicitis']
no_appendicitis = data_encoded[data_encoded['Diagnosis'] == 'no appendicitis']
len(appendicitis), len(no_appendicitis)

appendicitis = appendicitis[numerical_features]
no_appendicitis = no_appendicitis[numerical_features]


results = []
for feat in numerical_features:
    #temperarily drop NA values
    appendicitis = appendicitis.dropna(subset=[feat])
    no_appendicitis = no_appendicitis.dropna(subset=[feat])
    if len(appendicitis) == 0 or len(no_appendicitis) == 0:
        continue
    t_test = mannwhitneyu(appendicitis[feat], no_appendicitis[feat], alternative="two-sided")
    results.append([feat, t_test.statistic, t_test.pvalue])

num_feat_diagnosis = pd.DataFrame(results, columns=["feature", "statistic", "pvalue"])

sorted_num_feat_diagnosis = num_feat_diagnosis.sort_values(by="statistic", ascending=False)

display(sorted_num_feat_diagnosis)

#bonfroni correction
signifigance = 0.05 / len(numerical_features)
display(signifigance)

num_feat_diagnosis = sorted_num_feat_diagnosis[sorted_num_feat_diagnosis['pvalue'] < signifigance]
num_feat_diagnosis

In [ ]:
for col in data_encoded.columns:
    if 'Management' in col:
        print(col)


In [ ]:
#Same thing but now predicting the other targets
from scipy.stats import kruskal

#management
conservative = data_encoded[data_encoded['Management'] == "conservative"]
primary_surgical = data_encoded[data_encoded['Management'] == "primary surgical"]
secondary_surgical = data_encoded[data_encoded['Management'] == "secondary surgical"]
simultaneous_appendectomy = data_encoded[data_encoded['Management'] == "simultaneous appendectomy"]
len(conservative), len(primary_surgical), len(secondary_surgical), len(simultaneous_appendectomy)

conservative = conservative[numerical_features]
primary_surgical = primary_surgical[numerical_features]
secondary_surgical = secondary_surgical[numerical_features]
simultaneous_appendectomy = simultaneous_appendectomy[numerical_features]
len(conservative), len(primary_surgical), len(secondary_surgical), len(simultaneous_appendectomy)
#we won't use simultaneous appendectomy because there's only 1 value

results = []
for feat in numerical_features:
    #temperarily drop NA values
    conservative = conservative.dropna(subset=[feat])
    primary_surgical = primary_surgical.dropna(subset=[feat])
    secondary_surgical = secondary_surgical.dropna(subset=[feat])
    if len(conservative) == 0 or len(primary_surgical) == 0 or len(secondary_surgical) == 0:
        continue
    t_test = kruskal(conservative[feat], primary_surgical[feat], secondary_surgical[feat])
    results.append([feat, t_test.statistic, t_test.pvalue])

num_feat_management = pd.DataFrame(results, columns=["feature", "statistic", "pvalue"])
display(num_feat_management)

#bonfroni correction
signifigance = 0.05 / len(numerical_features)
display(signifigance)

num_feat_management = num_feat_management[num_feat_management['pvalue'] < signifigance]
num_feat_management

In [ ]:
#severity
unique_severity = data_encoded['Severity'].unique()
print(unique_severity)

complicated = data_encoded[data_encoded['Severity'] == "complicated"]
uncomplicated = data_encoded[data_encoded['Severity'] == "uncomplicated"]
display(len(complicated), len(uncomplicated))

complicated = complicated[numerical_features]
uncomplicated = uncomplicated[numerical_features]


results = []
for feat in numerical_features:
    #temperarily drop NA values
    complicated = complicated.dropna(subset=[feat])
    uncomplicated = uncomplicated.dropna(subset=[feat])
    if len(complicated) == 0 or len(uncomplicated) == 0:
        continue
    t_test = mannwhitneyu(complicated[feat], uncomplicated[feat], alternative="two-sided")
    results.append([feat, t_test.statistic, t_test.pvalue])

num_feat_severity = pd.DataFrame(results, columns=["feature", "statistic", "pvalue"])
display(num_feat_severity)

#bonfroni correction
signifigance = 0.05 / len(numerical_features)
display(signifigance)

num_feat_severity = num_feat_severity[num_feat_severity['pvalue'] < signifigance]
num_feat_severity

Now for categotrical features

In [92]:
# from scipy.stats import pearsonr

# COL = 'Diagnosis_no appendicitis'

# # Calculate Pearson correlation coefficients and p-values
# correlation_results = []
# for feature in numerical_features:
#     corr, p_value = pearsonr(data_encoded[feature], data_encoded[COL])
#     correlation_results.append((feature, corr, p_value))

# # Convert results to DataFrame
# correlation_df = pd.DataFrame(correlation_results, columns=['feature', 'correlation', 'pvalue'])

# # Sort by absolute correlation value
# correlation_df['abs_correlation'] = correlation_df['correlation'].abs()
# correlation_df = correlation_df.sort_values(by='abs_correlation', ascending=False)

# # filter out high p-values
# correlation_df = correlation_df[correlation_df['pvalue'] < 0.05/len(numerical_features)]

# # Drop the Diagnoses column
# correlation_df = correlation_df[correlation_df['feature'] != COL]


# # Print the sorted features, correlations, and p-values
# for i,row in enumerate(correlation_df.itertuples()):
#     print(f'{i}: {row.feature}: {row.correlation:.4f} (p-value: {row.pvalue:.4f})')

# # select the top 5 features

# top_features = correlation_df['feature'].values[:10]
# top_features

In [93]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report



def GaussianNB_train(features_to_use,y_col):

    data_encoded_no_na = data_encoded.dropna(subset=features_to_use)
    X = data_encoded_no_na[features_to_use]
    y = data_encoded_no_na[y_col]

    display(X.head(1))

    mean_report = {
        'accuracy': 0,
        'precision': 0,
        'recall': 0,
        'f1-score': 0
    }

    TESTS=1000

    for i in range(TESTS):

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        gnb = GaussianNB()

        gnb.fit(X_train, y_train)

        y_pred = gnb.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

        mean_report['accuracy'] += accuracy
        mean_report['precision'] += report['weighted avg']['precision']
        mean_report['recall'] += report['weighted avg']['recall']
        mean_report['f1-score'] += report['weighted avg']['f1-score']


    for key in mean_report:
        mean_report[key] /= TESTS

    print(mean_report)

In [ ]:
GaussianNB_train(num_feat_diagnosis['feature'], 'Diagnosis')

In [ ]:
GaussianNB_train(num_feat_severity['feature'], 'Severity')

In [ ]:
GaussianNB_train(num_feat_management['feature'], 'Management')